In [0]:
%run "./init"

In [0]:
import mlflow
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

client = mlflow.tracking.MlflowClient()
experiment = client.get_experiment_by_name(experiment_name)

if experiment is None:
    raise ValueError("Experiment does not exist. Please run notebook 1 to create the experiment.")

runs = client.search_runs(experiment.experiment_id)

df = pd.DataFrame([{
    "scenario": r.data.params.get("scenario"),
    "factory_down": r.data.params.get("factory_down"),
    "cost": r.data.metrics.get("total_cost"),
    "unmet": r.data.metrics.get("total_unmet_demand")
} for r in runs])
baseline_cost = df[df["scenario"] == "Baseline"]["cost"].values[0]
df["regret"] = df["cost"] - baseline_cost

df.sort_values("regret", ascending=False)
df

In [0]:
plt.figure(figsize=(10,6))
sns.barplot(x="scenario", y="regret", data=df)
plt.title("Regret per Disruption Scenario")
plt.ylabel("Cost Increase vs. Baseline")
plt.xlabel("Scenario")
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.savefig("regret_barplot.png", dpi=300)
plt.show()

In [0]:
pivot = df.pivot(index="factory_down", columns="scenario", values="unmet").fillna(0)
plt.figure(figsize=(8, 5))
sns.heatmap(pivot, annot=True, cmap="Reds", cbar_kws={'label': 'Unmet Demand'})
plt.title("Unmet Demand by Factory Disruption")
plt.tight_layout()
plt.savefig("unmet_heatmap.png", dpi=300)
plt.show()

In [0]:
def to_markdown_table(df):
    return df.to_markdown(index=False, tablefmt="github")

print(to_markdown_table(df[["scenario", "factory_down", "cost", "regret", "unmet"]]))

In [0]:
df.to_csv("disruption_scenario_results.csv", index=False)
pivot.to_csv("unmet_heatmap_data.csv")